<a href="https://colab.research.google.com/github/RafaelCaballero/Julio25/blob/main/code/proyecto_clasificacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción a la ciencia de datos con Python
Rafael Caballero

## Proyecto clasificación

Empezamos haciendo la carga y preprocesado, que ya explicamos en detalle en el notebook de regresión

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RepeatedKFold, cross_validate
from sklearn.metrics import mean_squared_error, make_scorer
from math import sqrt
import scipy.stats as stats
import numpy as np
import matplotlib.pyplot as plt
from datetime import date


import yfinance as yf
import pandas as pd

# devuelve los valores Close desde la fecha de inicio hasta hoy
# de todos los valoes que se le indican
def carga(valores, tickers, start_date='2022-01-01'):
  # Fechas de inicio y fin
  start_date = '2022-01-01'
  hoy = date.today()
  end_date = hoy.strftime("%Y-%m-%d")

  # Descargar los datos de Yahoo Finance
  data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')

  # Crear un DataFrame vacío para almacenar los valores CLOSE
  close_data = pd.DataFrame()

  # Extraer los valores OPEN de cada ticker y añadirlos al DataFrame
  for i,ticker in enumerate(tickers):
      nombre = valores[i]
      close_data[nombre] = data[ticker]['Close']
  return close_data


# añade los rendimientos en incremento sobre 100 para cada valor
# los rendimientos son columnas nuevas que se añaden al final
def ROI(df):
  # incremento en porcentaje con respecto al día anterior
  roi = df.pct_change()*100
  roi.columns = ["r_"+c for c in df.columns]
  df2 = pd.concat([df,roi],axis=1)
  # Establecer el índice como una versión formateada de la fecha
  df.index = pd.to_datetime(df.index).strftime('%Y-%m-%d')
  return df2.dropna()

def dia_semana(df):
  # añadimos el día de la semana
  df['dia_semana'] = roi.index.to_series().dt.day_name()

  # mejor en castellano
  days_translation = {
      'Monday': 'Lunes',
      'Tuesday': 'Martes',
      'Wednesday': 'Miércoles',
      'Thursday': 'Jueves',
      'Friday': 'Viernes',
      'Saturday': 'Sábado',
      'Sunday': 'Domingo'
  }
  df['dia_semana'] = df['dia_semana'].map(days_translation)
  return pd.get_dummies(df)


# sube la columna indicada c la cantidad de pasos indicada para igualarla
# con la columna con los datos del día actual
def columna_mañana(df,c,dias=1):
  df[c+"_futuro"] = df[c].shift(-dias)
  # al hacerlo han quedado nulos al final, los días de los que no tenemos el fut.
  df = df.iloc[:-dias]
  return df





In [ ]:
# Lista de tickers:
valores = ["inditex","iberdrola", "santander", "BBVA", "naturgy"]
tickers = ['ITX.MC', 'IBE.MC',    'SAN.MC', 'BBVA.MC', 'NTGY.MC']
df_raw = carga(valores,tickers)

df_roi = ROI(df_raw)
df_regresion = columna_mañana(df_roi,"r_BBVA")

df_regresion

Aun queda un último paso, convertir el problema de regresión en uno de clasificación

In [ ]:
df_regresion["r_BBVA_futuro"].hist()

In [ ]:
filtro = df_regresion["r_BBVA_futuro"] > 1 # subidas mayores
filtro.mean()

In [ ]:
df = df_regresion.copy()
df = df.drop(columns=["r_BBVA_futuro"])
df["label"] = 0  # se empieza con un valor común, luego se adapta
df.loc[filtro,"label"] = 1

In [ ]:
df

## Clasificación

Ahora empieza la clasificación en sí

Para ello vamos a definir primero la X y la Y

In [ ]:
yColumn = "label"
XColumns = [c for c in df.columns if c!="label" and c.startswith("r_")]
#XColumns = [ 'r_iberdrola',  'r_BBVA', 'r_naturgy']
y = df[yColumn]
X = df[XColumns]

In [ ]:
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.metrics import cohen_kappa_score, make_scorer
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")

#
steps = [
    ('scale', StandardScaler(with_mean=False)),
    ('over', RandomOverSampler()), # para equilibrar, hay que probar otras posibilidades
    ('logistic', LogisticRegression(max_iter=10000))
]
pipeline = Pipeline(steps)

# aquí la val. cruzada
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=20)

kappa = make_scorer(cohen_kappa_score)

scores = cross_val_score(pipeline, X, y, scoring=kappa, cv=cv)

print(f"Cohen’s κ (media ± desv): {scores.mean():.3f} ± {scores.std():.3f}")